In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time


#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()


momentum_query = '''
CREATE TABLE momentum (
    Ticker TEXT NOT NULL,
    Price FLOAT,
    y1_Price Return FLOAT, 
    y1_percentile FLOAT, 
    m6_Price_Return FLOAT, 
    m6_percentile FLOAT,
    m3_Price_Return FLOAT, 
    m3_percentile FLOAT, 
    m1_Price_Return FLOAT, 
    m1_percentile FLOAT, 
    Weighted_Hurst_Exponent FLOAT
);
'''

cursor.execute(momentum_query)

conn.commit()


In [2]:
#calulate the Hurst exponent of a stock
def get_hurst_exponent(time_series, max_lag):
    """Returns the Hurst Exponent of the time series"""
        
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
momentum_cols = ['Ticker','Price',
                '1y Price Return','1y percentile', 
                '6m Price Return', '6m percentile', 
                '3m Price Return', '3m percentile',
                '1m Price Return','1m percentile',
                'Weighted Hurst Exponent']
momentum = pd.DataFrame(columns = momentum_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker

        info.sort_values("Date", ascending = True, inplace = True)
        info.set_index("Date", inplace = True)
        #Creating the df to be added to momentum
        y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
        m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
        m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
        m1 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=1))

        #momentum['Ticker'] = tick['0'][i]  
        try: 
            perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
            #momentum['1y Price Return'] = perc_change1y
            perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
            #momentum['6m Price Return'] = perc_change6m
            perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
            #momentum['3m Price Return'] = perc_change3m
            perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
            #momentum['1m Price Return'] = perc_change1m

            #momentum['Price'] = info['Close'] 
            #momentum['1y percentile'] = 0
            #momentum['6m percentile'] = 0
            #momentum['3m percentile'] = 0
            #momentum['1m percentile'] = 0
            #momentum['Weighted Hurst Exponent'] = 0
            t = [20, 100, 300, 500, 1000]
            total_hurst = 0
            weight = 0.4
            for j in range(len(t)): 
                #Use Weighted average to determine hurst exp of the stock
                hurst_exp = get_hurst_exponent(info["Close"].values, t[j])
                #print(tick["0"][i] + " Hurst exponent with " + str(t[j]) +  ' lags: ' + str(hurst_exp))
                total_hurst += hurst_exp*weight
                if j >= 2: 
                    weight = 0.1
                else: 
                    weight = 0.2
            #momentum = momentum.iloc[-1]
        except IndexError: 
            print('Index Error when producing momentum table')
            continue
        momentum_append = pd.Series([tick['0'][i],info['Close'][-1],perc_change1y,0,perc_change6m,0,perc_change3m,0,perc_change1m,0, total_hurst],
                                index = momentum_cols)
        #print(momentum_append)
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    stop = time.time()
    duration = stop-start
    print(duration)
    momentum.loc[len(momentum)] = momentum_append
#Get percentiles for each stock
cols = ['1y', '6m', '3m', '1m']
for c in cols: 
    for index, row in momentum.iterrows(): 
        percentile_change = stats.percentileofscore(momentum[c + ' Price Return'], momentum[c + ' Price Return'].loc[index])
        momentum[c + ' percentile'][index] = percentile_change 
momentum['Shares To Buy'] = 0
    
#Calculate HQM score
#Get the mean of all 4 percentiles 
momentum['HQM score'] = 0
from statistics import mean
for index, row in momentum.iterrows(): 
    all_periods_p = [row['1y percentile'], row['6m percentile'], row['3m percentile'], row['1m percentile']]
    momentum['HQM score'].iloc[index] = mean(all_periods_p)


#PUSH TO DATABASE
momentum.to_sql('momentum', conn, if_exists='append', index=False)

# Close the connection
conn.close()

<ipython-input-2-2a4a68363179>:37: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-2-2a4a68363179>:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-2-2a4a68363179>:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-2-2a4a68363179>:40: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

1.5908231735229492


<ipython-input-2-2a4a68363179>:37: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-2-2a4a68363179>:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-2-2a4a68363179>:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-2-2a4a68363179>:40: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

0.6784226894378662


<ipython-input-2-2a4a68363179>:37: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-2-2a4a68363179>:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-2-2a4a68363179>:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-2-2a4a68363179>:40: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

...

0.7018694877624512
ANW: 1d data not available for startTime=-2208994789 and endTime=1673491424. Only 100 years worth of day granularity data are allowed to be fetched per request.
Index Error when producing momentum table


<ipython-input-2-2a4a68363179>:37: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-2-2a4a68363179>:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-2-2a4a68363179>:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-2-2a4a68363179>:40: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

1.2095496654510498


<ipython-input-2-2a4a68363179>:37: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-2-2a4a68363179>:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-2-2a4a68363179>:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-2-2a4a68363179>:40: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

Index Error when producing momentum table


In [ ]:
momentum.info()